# 🌠 [Day 4](https://adventofcode.com/2020/day/4)

In [1]:
import re

def passport_check_keys(lines):
    pattern = re.compile("(byr|iyr|eyr|hgt|hcl|ecl|pid|cid):(\S+)( |\n|$)")
    valid_keys = sorted(['byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid'])
    num_valid_pwds = 0
    for line in lines:
        keys = [match.group(1) for match in re.finditer(pattern, line)
                if match.group(1) != 'cid']
        num_valid_pwds += int(sorted(keys) == valid_keys)
    return num_valid_pwds


def passport_check_format(lines):
    pattern = re.compile("(byr:[0-9]{4}"
                         "|iyr:[0-9]{4}"
                         "|eyr:[0-9]{4}"
                         "|hgt:[0-9]+(cm|in)"
                         "|hcl:#[a-f0-9]{6}"
                         "|ecl:(amb|blu|brn|gry|grn|hzl|oth)"
                         "|pid:[0-9]{9}"
                         "|cid:\S+)"
                         "( |\n|$)")
    valid_keys = ['byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid']
    pwds = []
    for line in lines:
        p = {match[0][:3]: match[0][4:] 
             for match in re.findall(pattern, line)}
        # Check that all keys are present
        valid = True
        for k in valid_keys:
            if k not in p:
                valid = False
                break
        if not valid: continue
        # Check number ranges
        valid = ((1920 <= int(p['byr']) <= 2002) and
                 (2010 <= int(p['iyr']) <= 2020) and
                 (2020 <= int(p['eyr']) <= 2030) and
                 ((p['hgt'][-2:] == 'cm' and 150 <= int(p['hgt'][:-2]) <= 193) or
                  (p['hgt'][-2:] == 'in' and 59 <= int(p['hgt'][:-2]) <= 76))
                )
        if valid:
            pwds.append(p)
    return len(pwds), pwds

In [2]:
with open('inputs/day04.txt', 'r') as f:
    inputs = f.read().split('\n\n')
    
print(f"There are {passport_check_keys(inputs)} passports with valid keys")
print(f"But only {passport_check_format(inputs)[0]} valid passports")